# <center> ML-7. Прогнозирование биологического ответа (HW-3) </center>

### Условия задания
Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.            
Предварительная обработка не требуется, данные уже закодированы и нормализованы.           
В качестве метрики будем использовать F1-score.

In [16]:
# Импортируем библиотеки
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
from sklearn import linear_model #линейные моделиё
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

In [17]:
# Загружаем данные
data = pd.read_csv('../data/_train_sem09__1_.zip', compression='zip')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Выделяем матрицу наблюдений и столбец с правильными ответами
X = data.drop(['Activity'], axis=1)
y = data['Activity']

# Делим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=20, random_state=42)

In [19]:
# Создаем и обучаем модель логистической регрессии
model_lr1 = linear_model.LogisticRegression(max_iter=1000, random_state=42)
model_lr1.fit(X_train,y_train)

# Делаем предсказание и считаем метрику
y_test_pred = model_lr1.predict(X_test)
print(f'Значение метрики f1-score для модели логистической регрессии: {round(metrics.f1_score(y_test, y_test_pred),2)}')

Значение метрики f1-score для модели логистической регрессии: 0.75


In [20]:
# Создаем и обучаем модель случайного леса
model_rf1 = ensemble.RandomForestClassifier(random_state=42,)
model_rf1.fit(X_train,y_train)

# Делаем предсказание и считаем метрику
y_test_pred = model_rf1.predict(X_test)
print(f'Значение метрики f1-score для модели случайного леса: {round(metrics.f1_score(y_test, y_test_pred),2)}')

Значение метрики f1-score для модели случайного леса: 0.82


# <center> GridSearchCV </center>

### Логистическая регрессия

In [21]:
# Задаем сетку гиперпараметров для логистической регрессии
param_grid = [{'penalty':['l1', 'l2'],
               'solver':['liblinear', 'saga'],
               'C':[0.01, 0.3, 0.5, 0.7, 0.9]
               }
              ]

# Создаем объект класса GridSearchCV для модели логистической регрессии
grid_search_lr = GridSearchCV(estimator=linear_model.LogisticRegression(max_iter=5000,
                                                                        random_state=42
                                                                        ),
                              param_grid=param_grid,
                              cv=5,
                              n_jobs=-1,
                              scoring='f1'
                              )

# Производим подбор гиперпараметров
%time grid_search_lr.fit(X_train, y_train)

CPU times: user 1min 43s, sys: 104 ms, total: 1min 43s
Wall time: 7min 53s


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=5000, random_state=42),
             n_jobs=-1,
             param_grid=[{'C': [0.01, 0.3, 0.5, 0.7, 0.9],
                          'penalty': ['l1', 'l2'],
                          'solver': ['liblinear', 'saga']}],
             scoring='f1')

In [22]:
# Выведем лучшую комбинацию гиперпараметров и значение метрики на тестовой выборке
print(f'Лучшая комбинация гиперпараметров: {grid_search_lr.best_params_}')
print(f'Значение метрики f1-score на тестовой выборке для моделий логистической регрессии: {grid_search_lr.score(X_test, y_test)}')

Лучшая комбинация гиперпараметров: {'C': 0.5, 'penalty': 'l1', 'solver': 'saga'}
Значение метрики f1-score на тестовой выборке для моделий логистической регрессии: 0.75


### Случайный лес

In [110]:
# Задаем сетку гиперпараметров для случайного леса 
# параметры для поиска по сетке отличаются от других методов поиска 
# так как в задании введено ограничение - количество итераций не превышает 50
param_grid_rf = [{'criterion':['gini', 'entropy'], 
                  'n_estimators':[100, 200, 300],
                  'min_samples_leaf':[1,5,7],
                  'max_depth':[50, 100]                  
                  }
                 ]

# Создаем объект класса GridSearchCV для модели случайного леса
grid_search_rf = GridSearchCV(estimator=ensemble.RandomForestClassifier(random_state=42),
                              param_grid=param_grid_rf,
                              cv=5,
                              n_jobs=-1,
                              scoring='f1'
                              )

# Производим подбор гиперпараметров
%time grid_search_rf.fit(X_train, y_train)

CPU times: user 4.36 s, sys: 28.6 ms, total: 4.39 s
Wall time: 51.1 s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [50, 100], 'min_samples_leaf': [1, 5, 7],
                          'n_estimators': [100, 200, 300]}],
             scoring='f1')

In [111]:
# Выведем лучшую комбинацию гиперпараметров и значение метрики на тестовой выборке
print(f'Лучшая комбинация гиперпараметров: {grid_search_rf.best_params_}')
print(f'значение метрики f1-score на тестовой выборке для модели случайного леса: {round(grid_search_rf.score(X_test, y_test),2)}')

Лучшая комбинация гиперпараметров: {'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 1, 'n_estimators': 300}
значение метрики f1-score на тестовой выборке для модели случайного леса: 0.82


### Результат для способа подбора параметров GridSearchCV
Для модели логистической регрессии значение метрики f1-score - 0.75, время подбора параметров составило: 7 минут 53 секунды. Для модели случайного леса значение метрики f1-score - 0.82, время подбора параметров составило: 51 секунду. Метрику для обеих моделей улучшить не удалось.

# <center> RandomizedSearchCV </center>

### Логистическая регрессия

In [25]:
# Задаем сетку гиперпараметров для логистической регрессии
param_grid = [{'penalty':['l1', 'l2'],
               'solver':['liblinear', 'saga'],
               'C':list(np.linspace(0.01, 1, 10, dtype=float))
               }
              ]

# Создаем объект класса RandomizedSearchCV для модели логистической регрессии
random_search_lr = RandomizedSearchCV(estimator=linear_model.LogisticRegression(max_iter=5000,
                                                                                random_state=42
                                                                                ),
                                      param_distributions=param_grid,
                                      cv=5,
                                      n_iter=20,
                                      n_jobs=-1,
                                      scoring='f1'
                                      )

# Производим подбор гиперпараметров
%time random_search_lr.fit(X_train, y_train)

CPU times: user 427 ms, sys: 42.3 ms, total: 470 ms
Wall time: 6min 29s


RandomizedSearchCV(cv=5,
                   estimator=LogisticRegression(max_iter=5000, random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions=[{'C': [0.01, 0.12, 0.23, 0.34, 0.45,
                                               0.56, 0.67, 0.78, 0.89, 1.0],
                                         'penalty': ['l1', 'l2'],
                                         'solver': ['liblinear', 'saga']}],
                   scoring='f1')

In [26]:
# Выведем лучшую комбинацию гиперпараметров и значение метрики на тестовой выборке
print(f'Лучшая комбинация гиперпараметров: {random_search_lr.best_params_}')
print(f'Значение метрики f1-scoreна тестовой выборке для модели логистической регрессии: {random_search_lr.score(X_test, y_test)}')


Лучшая комбинация гиперпараметров: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.34}
Значение метрики f1-scoreна тестовой выборке для модели логистической регрессии: 0.75


### Случайный лес

In [107]:
# Задаем сетку гиперпараметров для случайного леса
param_grid_rf = [{'criterion':['gini', 'entropy'], 
                  'n_estimators':list(range(100, 500, 50)),
                  'min_samples_leaf':list(np.linspace(1, 5, 5, dtype=int)),
                  'max_depth':list(range(25, 100, 5))                                   
                  }
                 ]

# Создаем объект класса RandomizedSearchCV для модели случайного леса
random_search_rf = RandomizedSearchCV(estimator=ensemble.RandomForestClassifier(random_state=42),
                              param_distributions=param_grid_rf,
                              cv=5,
                              n_iter=20,
                              n_jobs=-1,
                              scoring='f1'
                              )

# Производим подбор гиперпараметров
%time random_search_rf.fit(X_train, y_train)

CPU times: user 4.1 s, sys: 39.9 ms, total: 4.14 s
Wall time: 35.5 s


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions=[{'criterion': ['gini', 'entropy'],
                                         'max_depth': [25, 30, 35, 40, 45, 50,
                                                       55, 60, 65, 70, 75, 80,
                                                       85, 90, 95],
                                         'min_samples_leaf': [1, 2, 3, 4, 5],
                                         'n_estimators': [100, 150, 200, 250,
                                                          300, 350, 400,
                                                          450]}],
                   scoring='f1')

In [108]:
# Выведем лучшую комбинацию гиперпараметров и значение метрики на тестовой выборке
print(f'Лучшая комбинация гиперпараметров: {random_search_rf.best_params_}')
print(f'Значение метрики f1-score на тестовой выборке для модели случайного леса: {round(random_search_rf.score(X_test, y_test),2)}')

Лучшая комбинация гиперпараметров: {'n_estimators': 300, 'min_samples_leaf': 1, 'max_depth': 65, 'criterion': 'entropy'}
Значение метрики f1-score на тестовой выборке для модели случайного леса: 0.82


### Результат для способа подбора параметров RandomizedSearchCV
Для модели логистической регрессии значение метрики f1-score - 0.75, время подбора параметров составило: 6 минут 29 секунд. Для модели случайного леса значение метрики f1-score - 0,82, время подбора параметров составило: 35.5 секунд. Метрика для обеих моделей не изменилась.

# <center> Hyperopt </center>

### Логистическая регрессия

In [129]:
# Задаем пространство гиперпараметров для модели логистической регрессии
space = {'penalty':hp.choice('penalty', ['l1', 'l2']),
         'solver':hp.choice('solver', ['liblinear', 'saga']),
         'C':hp.quniform('C', 0.01, 1, 0.01)
         }
                 
random_state = 42 # фиксируем случайное число

# Задаем функцию для минимизации
def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty':params['penalty'],
               'solver':params['solver'],
               'C':params['C']
               }
    model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=5000)
        
    # используем кросс-валидацию
    score = cross_val_score(model, X, y, cv=cv, scoring='f1', n_jobs=-1).mean()
    return -score # так как используем метрику f1-score - ставим знак минус

In [130]:
# Начинаем подбор гиперпараметров
%time 

trials = Trials() # используется для логирования результатов

best_lr=fmin(hyperopt_lr, # наша функция 
             space=space, # пространство гиперпараметров
             algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
             max_evals=20, # максимальное количество итераций
             trials=trials, # логирование результатов
             rstate=np.random.default_rng(random_state) # фиксируем для повторяемости результата
             )
print("Наилучшие значения гиперпараметров {}".format(best_lr))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs
100%|██████████| 20/20 [14:41<00:00, 44.09s/trial, best loss: -0.7936242351503495]
Наилучшие значения гиперпараметров {'C': 0.47000000000000003, 'penalty': 0, 'solver': 1}


In [131]:
# Задаем списки с гиперпараметрами, так как категориальные значения сохраняются в виде индексов
penalty = ['l1', 'l2']
solver = ['liblinear', 'saga']

# Рассчитаем точность для тестовой выборки
model_hyp_lr = linear_model.LogisticRegression(max_iter=5000,
                                               random_state=random_state, 
                                               penalty=penalty[best_lr['penalty']],
                                               solver=solver[best_lr['solver']],
                                               C=best_lr['C']
                                               )
model_hyp_lr.fit(X_train, y_train)
y_test_pred = model_hyp_lr.predict(X_test)
print('f1_score на тестовом наборе для модели логистической регрессии: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе для модели логистической регрессии: 0.75


### Случайный лес

In [99]:
# Задаем пространство гиперпараметров для модели случайного леса
space_rf = {'criterion':hp.choice('criterion', ['gini', 'entropy']),
            'n_estimators':hp.quniform('n_estimators', 100, 500, 5),
            'min_samples_leaf':hp.quniform('min_samples_leaf', 1, 5, 1),
            'max_depth':hp.quniform('max_depth', 25, 100, 5)  
            }
               
    
random_state = 42 # фиксируем случайное число

# Задаем функцию для минимизации
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'criterion':params['criterion'],
               'n_estimators':int(params['n_estimators']),
               'min_samples_leaf':int(params['min_samples_leaf']),
               'max_depth':int(params['max_depth'])
               }
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
        
    # используем кросс-валидацию
    score = cross_val_score(model, X, y, cv=cv, scoring='f1', n_jobs=-1).mean()
    return -score # так как используем метрику f1-score - ставим знак минус

In [124]:
# Начинаем подбор гиперпараметров
%time 

trials = Trials() # используется для логирования результатов

best_rf=fmin(hyperopt_rf, # наша функция 
             space=space_rf, # пространство гиперпараметров
             algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
             max_evals=20, # максимальное количество итераций
             trials=trials, # логирование результатов
             rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
             )
print("Наилучшие значения гиперпараметров {}".format(best_rf))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs
100%|██████████| 20/20 [01:06<00:00,  3.32s/trial, best loss: -0.8262375976440357]
Наилучшие значения гиперпараметров {'criterion': 1, 'max_depth': 60.0, 'min_samples_leaf': 2.0, 'n_estimators': 400.0}


In [125]:
# Задаем списки с гиперпараметрами, так как категориальные значения сохраняются в виде индексов
criterion = ['gini', 'entropy']

# Рассчитаем точность для тестовой выборки
model_hyp_rf = ensemble.RandomForestClassifier(random_state=random_state,
                                               criterion=criterion[best_rf['criterion']],
                                               n_estimators=int(best_rf['n_estimators']),
                                               min_samples_leaf=int(best_rf['min_samples_leaf']),
                                               max_depth=int(best_rf['max_depth'])                                               
                                               )
model_hyp_rf.fit(X_train, y_train)
y_test_pred_rf = model_hyp_rf.predict(X_test)
print('f1_score на тестовом наборе для модели случайного леса: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf)))

f1_score на тестовом наборе для модели случайного леса: 0.78


### Результат для способа подбора параметров Hyperopt
Для модели логистической регрессии значение метрики f1-score - 0.75, время подбора параметров составило: 14 минут 41.8 секунды. Для модели случайного леса значение метрики f1-score - 0.78, время подбора параметров составило: 1 минута 6.3 секунды. Метрика для логистической модели осталась неизменной, а для модели случайного леса снизилась.

# <center> Optuna </center>

### Логистическая регрессия

In [126]:
# Задаем функцию оптимизации
def optuna_lr(trial):
    # задаем пространства поиска гиперпараметров
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    C = trial.suggest_float(name='C', low=0.01, high=1, step=0.01)
    
    # создаем модель
    model = linear_model.LogisticRegression(penalty=penalty,
                                            solver=solver,
                                            C=C,
                                            random_state=random_state,
                                            max_iter=5000
                                            )
    # обучаем модель
    model.fit(X_train, y_train)
    
    # используем кросс-валидацию
    score = cross_val_score(model, X, y, cv=5, scoring='f1', n_jobs=-1).mean()
    return score

In [127]:
%%time
# cоздаем объект исследования
study_lr = optuna.create_study(study_name="LogisticRegression", direction="maximize")

# ищем лучшую комбинацию гиперпараметров n_trials раз
study_lr.optimize(optuna_lr, n_trials=20)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_lr.best_params))

[I 2023-06-19 23:06:26,468] A new study created in memory with name: LogisticRegression
[I 2023-06-19 23:06:41,735] Trial 0 finished with value: 0.7838901806138469 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.01}. Best is trial 0 with value: 0.7838901806138469.
[I 2023-06-19 23:06:43,385] Trial 1 finished with value: 0.7792650509409993 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.67}. Best is trial 0 with value: 0.7838901806138469.
[I 2023-06-19 23:07:36,437] Trial 2 finished with value: 0.7765511501982312 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.86}. Best is trial 0 with value: 0.7838901806138469.
[I 2023-06-19 23:08:05,774] Trial 3 finished with value: 0.7866743041280643 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.060000000000000005}. Best is trial 3 with value: 0.7866743041280643.
[I 2023-06-19 23:11:48,149] Trial 4 finished with value: 0.7812933485213779 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.53}.

Наилучшие значения гиперпараметров {'penalty': 'l1', 'solver': 'liblinear', 'C': 0.17}
CPU times: user 3min 29s, sys: 825 ms, total: 3min 30s
Wall time: 7min 3s


In [128]:
# рассчитаем f1 для тестовой выборки
model_lr_opt = linear_model.LogisticRegression(**study_lr.best_params,random_state=random_state, max_iter=2500)
model_lr_opt.fit(X_train, y_train)

# делаем предсказание и рассчитываем метрику
y_test_pred_lr_opt = model_lr_opt.predict(X_test)
print('f1_score на тестовом наборе для модели логистической регрессии: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr_opt)))

f1_score на тестовом наборе для модели логистической регрессии: 0.75


Случайный лес

In [104]:
# Задаем функцию оптимизации для моедли случайного леса
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 500, 5)
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5, 1)
  max_depth = trial.suggest_int('max_depth', 25, 100, 5)  

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          criterion=criterion,
                                          min_samples_leaf=min_samples_leaf,
                                          max_depth=max_depth,
                                          random_state=random_state)
  # обучаем модель с кросс-валидацией 
  score = cross_val_score(model, X, y, cv=5, scoring='f1', n_jobs=-1).mean()

  return score

In [105]:
%%time
# cоздаем объект исследования
study_rf = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")

# ищем лучшую комбинацию гиперпараметров n_trials раз
study_rf.optimize(optuna_rf, n_trials=20)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study_rf.best_params))

[I 2023-06-19 21:50:36,182] A new study created in memory with name: RandomForestClassifier
[I 2023-06-19 21:50:38,909] Trial 0 finished with value: 0.8124887079994556 and parameters: {'n_estimators': 205, 'criterion': 'gini', 'min_samples_leaf': 1, 'max_depth': 80}. Best is trial 0 with value: 0.8124887079994556.
[I 2023-06-19 21:50:42,529] Trial 1 finished with value: 0.8154282739186783 and parameters: {'n_estimators': 295, 'criterion': 'gini', 'min_samples_leaf': 1, 'max_depth': 30}. Best is trial 1 with value: 0.8154282739186783.
[I 2023-06-19 21:50:47,716] Trial 2 finished with value: 0.8171690797258151 and parameters: {'n_estimators': 405, 'criterion': 'entropy', 'min_samples_leaf': 2, 'max_depth': 85}. Best is trial 2 with value: 0.8171690797258151.
[I 2023-06-19 21:50:51,673] Trial 3 finished with value: 0.8120016454567596 and parameters: {'n_estimators': 375, 'criterion': 'gini', 'min_samples_leaf': 4, 'max_depth': 40}. Best is trial 2 with value: 0.8171690797258151.
[I 2023-0

Наилучшие значения гиперпараметров {'n_estimators': 495, 'criterion': 'entropy', 'min_samples_leaf': 2, 'max_depth': 100}
CPU times: user 370 ms, sys: 255 ms, total: 625 ms
Wall time: 1min 25s


In [106]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study_rf.best_params,random_state=random_state, )
model.fit(X_train, y_train)

# делаем предсказание и рассчитываем метрику
y_test_pred_rf_opt = model.predict(X_test)
print('f1_score на тестовом наборе для модели случайного леса: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_opt)))

f1_score на тестовом наборе для модели случайного леса: 0.82


### Результат для способа подбора параметров Optuna
 Для модели логистической регрессии значение метрики f1-score - 0.75 , время подбора параметров составило: 7 минут 3 секунды. Для модели случайного леса значение метрики f1-score - 0.82 , время подбора параметров составило: 1 минуту 25 секунд.

### Выводы
1. Время работы сильно зависист от скорости обучения модели что в свою очередь зависит от заданных в сетке поиска параметров. К примеру увеличение количества деревьев увеличивает общее время подбора параметров. А для модели логистической регрессии из-за выбранных параметров регуляризации пришлось задать количество итераций на сходимость - 5000, иначе при некоторых сочетаниях параметров модель не сходилась. Это очень сильно увеличило время подбора параметров.
2. Во всех попытках подбора кроме одной метрика осталась прежней, можно предположить, что данные хорошо подготовлены поэтому модель с базовыми настройками дает хороший результат. Исключение составляет подбор параметров способом Hyperopt для модели случайного леса, где метрика снизилась относительно базовой модели. Попытки увеличить количество итераций (до 150) не дали результата. Возможно это связано с особенностями алгоритма поиска и заданым пространством параметров.
3. Что касается времени работы, то при применении кросс-валидации последние два способа даже проигрывают по времени способу случайного поиска. Видимо это связано с тем, что последние два способа тратят дополнительное время на вычисление следующей оптимальной комбинации. Тогда как случайный поиск использует случайный набор параметров.
4. Интересная особенность при задании диапзона параметра С, для модели логистической регрессии, в виде диапазона от 0.01 до 1 с шагом 0.1, в способе Optuna выскакивало предупреждение, что диапазон не делится ровно на шаг и будет заменен на диапазон от 0.01 до 0.91. При этом время подбора параметров составляло для Optuna - 36 с лишним минут, а для Hyperopt - 25 минут. Лучшее значение параметра С в обоих случаях представляло число с большим количеством знаков после запятой. Изменение шага на 0.01 позволило сократить время подбора параметров для Optuna до 7 минут 3 секунд, а для Hyperopt до 14 минут 41 секунды. Однако лучшее значение параметра С для Hyperopt все равно имеет вид 0.47000000000000003. Возможно параметр q, определяющий шаг в hp.quniform(label, low, high, q), плохо работает с дробными значениями.
5. В целом рассмотренные способы сильно зависят от заданного пространства поиска параметров. Слишком большое пространство даже с продвинутыми способами оптимизации могут занять много времени. Поэтому  стоит предварительно сократить пространство поиска, исключив объективно проигрышные комбинации.